# Custom SQL filter

**QuackOSM** enables advanced users to filter data using SQL filters that will be used by DuckDB during processing.

The filter will be loaded alongside with [OSM tags filters](../osm_tags_filter/) and features IDs filters. 

SQL filter clause will can be passed both in Python API (as `custom_sql_filter` parameter) and the CLI (as `--custom-sql-filter` option).

Two columns available to users are: `id` (type `BIGINT`) and `tags` (type: `MAP(VARCHAR, VARCHAR)`).

You can look for available functions into a [DuckDB documentation](https://duckdb.org/docs/sql/functions/overview).

Below are few examples on how to use the custom SQL filters.

## Features with exactly 10 tags

Here we will use `cardinality` function dedicated to the `MAP` type.

More `MAP` functions are available [here](https://duckdb.org/docs/sql/functions/map).

In [ ]:
import quackosm as qosm

data = qosm.convert_geometry_to_geodataframe(
    geometry_filter=qosm.geocode_to_geometry("Greater London"),
    osm_extract_source="Geofabrik",
    custom_sql_filter="cardinality(tags) = 10",
)
data["tags"].head(10).values

In [ ]:
print("All have exactly 10 tags:", (data["tags"].str.len() == 10).all())

## Features with ID divisible by 13 and starting wit a number 6

Here we will operate on the `ID` column.

More `NUMERIC` functions are available [here](https://duckdb.org/docs/sql/functions/numeric).

More `STRING` functions are available [here](https://duckdb.org/docs/sql/functions/char).

In [ ]:
data = qosm.convert_geometry_to_geodataframe(
    geometry_filter=qosm.geocode_to_geometry("Greater London"),
    osm_extract_source="Geofabrik",
    custom_sql_filter="id % 13 = 0 AND starts_with(id::STRING, '6')",
)
data

In [ ]:
print("All starting with digit 6:", data.index.map(lambda x: x.split("/")[1].startswith("6")).all())
print("All divisible by 13:", data.index.map(lambda x: (int(x.split("/")[1]) % 13) == 0).all())

## Find features that have all selected tags present

When using `osm_tags_filter` with value `{ "building": True, "historic": True, "name": True }`, the result will contain every feature that have at least one of those tags.

Positive tags filters are combined using an `OR` operator. You can read more about it [here](../osm_tags_filter/).

To get filters with `AND` operator, the `custom_sql_filter` parameter has to be used.

To match a list of keys against given values we have to use list-related functions.

More `LIST` functions are available [here](https://duckdb.org/docs/sql/functions/list).

In [ ]:
data = qosm.convert_geometry_to_geodataframe(
    geometry_filter=qosm.geocode_to_geometry("Greater London"),
    osm_extract_source="Geofabrik",
    custom_sql_filter="list_has_all(map_keys(tags), ['building', 'historic', 'name'])",
)
data

In [ ]:
tags_names = ["name", "building", "historic"]
for tag_name in tags_names:
    data[tag_name] = data["tags"].apply(lambda x, tag_name=tag_name: x.get(tag_name))
data[[*tags_names, "geometry"]].explore(tiles="CartoDB DarkMatter", color="orange")

## Regex search to find streets starting with word New or Old

*(If you really need to)* You can utilize regular expressions on a tag value (or key) to find some specific examples.

More `REGEX` functions are available [here](https://duckdb.org/docs/sql/functions/regular_expressions).

In [ ]:
data = qosm.convert_geometry_to_geodataframe(
    geometry_filter=qosm.geocode_to_geometry("Greater London"),
    osm_extract_source="Geofabrik",
    custom_sql_filter="""
        list_has_all(map_keys(tags), ['highway', 'name'])
        AND regexp_matches(tags['name'], '^(New|Old)\s\w+')
    """,
)
data

In [ ]:
ways_only = data[data.index.str.startswith("way/")]
ways_only["name"] = ways_only["tags"].apply(lambda x: x["name"])
ways_only["prefix"] = ways_only["name"].apply(lambda x: x.split()[0])
ways_only[["name", "prefix", "geometry"]].explore(
    tiles="CartoDB DarkMatter", column="prefix", cmap=["orange", "royalblue"]
)